In [2]:
import mesh_utils
import numpy as np
import importlib
importlib.reload(mesh_utils)

<module 'mesh_utils' from '/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/tools/mesh_utils.py'>

## For converting the raw .vtp and .vtu files to openCARP format:

In [4]:
# data_path = "../data/instance_001/"
# vtp_path = data_path + "instance_001.vtp"
# vtu_path = data_path + "instance_001.vtu"
# output_prefix = "instance_001"

# mesh_utils.vtk_to_opencarp(vtp_path, vtu_path, output_prefix, data_path)

## Read in the points, surfaces, tetrahedra, and UVCs

In [5]:
prefix = "instance_001"
data_path = f"../data/{prefix}"

reader = mesh_utils.OpenCARPMeshReader(data_path, prefix)
points, tetrahedra, tetrahedra_regions, triangles, triangle_regions, uvc_data = reader.read_all()

Reading points from ../data/instance_001/instance_001.pts
Loaded 478820 points with shape (478820, 3)
Reading tetrahedra from ../data/instance_001/instance_001.elem
Loaded 2555157 tetrahedra with shape (2555157, 4)
Reading triangles from ../data/instance_001/instance_001.surf
Loaded 199902 triangles with shape (199902, 3)
Reading UVC data from ../data/instance_001/instance_001_UVC.csv
Loaded UVC data with shape (478820, 6)
UVC columns: ['tv', 'tm', 'rtSin', 'rtCos', 'rt', 'ab']


## Use the UVCs to define a coordinate system and compute the fiber and sheet directions

In [6]:
phi_transmural = np.array(uvc_data['tm'])
phi_longitudinal = np.array(uvc_data['ab'])
phi_circumferential = np.array(uvc_data["rt"])

In [7]:
TransmuralField, LongitudinalField, CircumferentialField = mesh_utils.compute_normed_gradients(points, -phi_transmural, -phi_longitudinal, phi_circumferential)

mesh_utils.visualize_vector_fields(
    points, triangles, triangle_regions,
    TransmuralField, LongitudinalField, CircumferentialField, subsample_factor=50, glyph_scale=5000
)

Original points: 478820
Original triangles: 199902
Epicardium triangles: 91160
Endocardium triangles: 104504
Sampled epicardium points: 917
Sampled endocardium points: 1053


Widget(value='<iframe src="http://localhost:59447/index.html?ui=P_0x12dd24a90_0&reconnect=auto" class="pyvista…

In [8]:
fiber_dirs, sheet_dirs, sheet_normal_dirs = mesh_utils.compute_fiber_sheet_directions(
    TransmuralField, LongitudinalField, CircumferentialField,
    phi_transmural, 
    endo_fiber_angle=60.0, epi_fiber_angle=-60, endo_sheet_angle=-65, epi_sheet_angle=25
)

mesh_utils.visualize_fibers(
    points, triangles, triangle_regions,
    fiber_dirs, subsample_factor=50, glyph_scale=5000
)

Original points: 478820
Original triangles: 199902
Epicardium triangles: 91160
Endocardium triangles: 104504
Sampled epicardium points: 917
Sampled endocardium points: 1053


Widget(value='<iframe src="http://localhost:59447/index.html?ui=P_0x1570b8280_1&reconnect=auto" class="pyvista…

In [9]:
mesh_utils.write_lon_file(
    f"{data_path}/{prefix}.lon", 
    fiber_dirs, 
    sheet_dirs, 
    sheet_normal_dirs,
    tetrahedra
)

Processing 2555157 elements for .lon file...


Precomputing element directions: 100%|██████████| 2555157/2555157 [00:50<00:00, 50584.42it/s]


Verifying orthogonality of element directions...
Maximum dot products: fiber·sheet=0.999987, fiber·normal=0.999816, sheet·normal=0.999924
Writing to ../data/instance_001/instance_001.lon...


Writing .lon file: 100%|██████████| 256/256 [00:05<00:00, 47.63it/s]

Successfully wrote fiber orientations to ../data/instance_001/instance_001.lon


## Tag the fast conducting endocardium

In [24]:
import numpy as np
from tqdm import tqdm

def tag_lcx_infarct_and_endocardium(
    points, 
    tetrahedra, 
    triangles, 
    triangle_regions, 
    phi_transmural,
    phi_longitudinal, 
    phi_circumferential,
    output_file,
    long_min=0.1,
    long_max=0.9,
    # Infarct location parameters
    infarct_center_circ=0.3,      # Center position around circumference (0-1, 0.25-0.35 for lateral wall)
    infarct_center_long=0.45,     # Center position along long axis (0-1, 0=apex, 1=base)
    # Infarct size parameters
    infarct_width_circ=0.06,      # Half-width in circumferential direction
    infarct_width_long=0.15,      # Half-width in longitudinal direction
    # Transmural extent parameters
    infarct_transmural_min=0.1,   # Minimum transmural extent (0=epicardium)
    infarct_transmural_max=0.7,   # Maximum transmural extent (1=endocardium)
    # Border zone parameters
    border_zone_factor=1.4,       # How much larger the border zone is compared to core
    border_transmural_max=0.9,    # Maximum transmural extent for border zone
    # Shape modifiers
    transmural_taper=0.5,         # How much the infarct tapers from epi to endo (0=no taper, 1=extreme taper)
    irregularity=1.0,             # Scale factor for boundary irregularity (0=smooth, 2=very irregular)
    random_seed=42,               # Seed for reproducible randomness
):
    """
    Tag tetrahedra for:
    1. Fast conducting endocardium (region 1)
    2. Core infarct zone in lateral LV wall (LCx territory) (region 2)
    3. Border zone around the infarct (region 3)
    
    Parameters:
    -----------
    points : ndarray (n_pts x 3)
        The 3D coordinates for every point in the cardiac mesh
    tetrahedra : ndarray (n_tets x 4)
        Indices of the four points in each tetrahedron
    triangles : ndarray (n_tris x 3)
        Indices of the three points in each surface triangle
    triangle_regions : ndarray (n_tris)
        Region markers for surface triangles
    phi_transmural : ndarray (n_pts)
        Transmural coordinate (1 on endocardium, 0 on epicardium)
    phi_longitudinal : ndarray (n_pts)
        Longitudinal coordinate (1 at base, 0 at apex)
    phi_circumferential : ndarray (n_pts)
        Circumferential coordinate (0 mod 1 at anterior septal wall)
    output_file : str
        Path to write the resulting .elem file
    long_min : float, optional
        Minimum longitudinal coordinate for fast conducting endocardium
    long_max : float, optional
        Maximum longitudinal coordinate for fast conducting endocardium
    infarct_center_circ : float, optional
        Center position around circumference (0-1, 0.25-0.35 for lateral wall)
    infarct_center_long : float, optional
        Center position along long axis (0-1, 0=apex, 1=base)
    infarct_width_circ : float, optional
        Half-width in circumferential direction
    infarct_width_long : float, optional
        Half-width in longitudinal direction
    infarct_transmural_min : float, optional
        Minimum transmural extent (0=epicardium)
    infarct_transmural_max : float, optional
        Maximum transmural extent (1=endocardium)
    border_zone_factor : float, optional
        How much larger the border zone is compared to core
    border_transmural_max : float, optional
        Maximum transmural extent for border zone
    transmural_taper : float, optional
        How much the infarct tapers from epi to endo (0=no taper, 1=extreme taper)
    irregularity : float, optional
        Scale factor for boundary irregularity (0=smooth, 2=very irregular)
    random_seed : int, optional
        Seed for reproducible randomness
    """
    print("Starting LAD infarct and endocardium tagging...")
    
    # Step 1: Identify points for fast conducting endocardium
    print(f"Identifying points with phi_longitudinal between {long_min} and {long_max}...")
    longitudinal_mask = (phi_longitudinal >= long_min) & (phi_longitudinal <= long_max)
    valid_longitudinal_points = set(np.where(longitudinal_mask)[0])
    print(f"Found {len(valid_longitudinal_points)} points with valid longitudinal coordinate")
    
    # Step 2: Identify points that are part of endocardial triangles (regions 3 or 4)
    print("Identifying endocardial points...")
    endocardial_triangles = np.where((triangle_regions == 3) | (triangle_regions == 4))[0]
    endocardial_points = set()
    for tri_idx in endocardial_triangles:
        for point_idx in triangles[tri_idx]:
            endocardial_points.add(point_idx)
    print(f"Found {len(endocardial_points)} points on the endocardium")
    
    # Step 3: Find intersection for endocardial conduction system
    valid_endo_points = valid_longitudinal_points.intersection(endocardial_points)
    print(f"Found {len(valid_endo_points)} points for fast conducting endocardium")
    
    # Step 4: Define LCx infarct region
    # Lateral LV wall, typically in the LCx territory
    print("Defining lateral wall (LCx) infarct region...")
    
    # First, identify LV vs RV points using the endocardial triangles
    print("Identifying LV points...")
    
    # Find LV endocardial triangles (region 3)
    lv_endo_triangles = np.where(triangle_regions == 3)[0]
    lv_endo_points = set()
    for tri_idx in lv_endo_triangles:
        for point_idx in triangles[tri_idx]:
            lv_endo_points.add(point_idx)
    print(f"Found {len(lv_endo_points)} points on the LV endocardium")
    
    # Find RV endocardial triangles (region 4)
    rv_endo_triangles = np.where(triangle_regions == 4)[0]
    rv_endo_points = set()
    for tri_idx in rv_endo_triangles:
        for point_idx in triangles[tri_idx]:
            rv_endo_points.add(point_idx)
    print(f"Found {len(rv_endo_points)} points on the RV endocardium")
    
    # Create a vector from endocardium to each point to determine LV vs RV association
    # This is a more robust way to determine ventricle association than just endocardial points
    
    # First, compute average position of LV and RV endocardial points
    lv_center = np.mean(points[list(lv_endo_points)], axis=0)
    rv_center = np.mean(points[list(rv_endo_points)], axis=0)
    print(f"LV center: {lv_center}")
    print(f"RV center: {rv_center}")
    
    # Vectorized distance calculation to LV and RV centers
    distances_to_lv = np.linalg.norm(points - lv_center, axis=1)
    distances_to_rv = np.linalg.norm(points - rv_center, axis=1)
    
    # Points closer to LV center than RV center are considered LV points
    lv_mask = distances_to_lv <= distances_to_rv
    print(f"Identified {np.sum(lv_mask)} points as LV territory")
    
    # LAD territory is typically:
    # - Anterior wall (phi_circumferential around 0.0 or 1.0)
    # - From mid to apical portion (phi_longitudinal in lower range)
    # - Can extend from epicardium to endocardium
    # - Only in the LV territory
    
    # Core infarct region with more natural, irregular shape
    # Using the provided parameters for center, size, and shape
    
    # Calculate "distance" from center point in UVC space
    # Using modified elliptical distance with randomization to create irregularity
    circ_dist = np.abs(phi_circumferential - infarct_center_circ)
    # Adjust for circular nature of circumferential coordinate
    circ_dist = np.minimum(circ_dist, 1.0 - circ_dist)
    long_dist = np.abs(phi_longitudinal - infarct_center_long)
    
    # Create irregular edges using sine functions to modulate the boundary
    np.random.seed(random_seed)  # For reproducibility
    num_points = len(phi_circumferential)
    
    # Generate random noise values (once, for efficiency)
    noise_factor = 0.02 * irregularity
    random_noise = np.random.normal(0, noise_factor, num_points)
    
    # Create wedge shape - wider at epicardium, narrower at endocardium
    # Scale the allowed circumferential width based on transmural depth
    transmural_factor = 1.0 - transmural_taper * phi_transmural  # Tapers toward endocardium
    
    # Modulate boundary with sine waves to create irregularity
    boundary_modulation = irregularity * (0.02 * np.sin(phi_longitudinal * 10) + 
                                         0.015 * np.sin(phi_circumferential * 12))
    
    # Combine factors to create irregular elliptical distance
    scaled_circ_dist = circ_dist / (infarct_width_circ * transmural_factor + boundary_modulation + random_noise)
    scaled_long_dist = long_dist / (infarct_width_long + 0.02 * irregularity * np.sin(phi_circumferential * 8) + random_noise)
    
    # Elliptical distance metric with asymmetry (infarct extends more in longitudinal direction)
    distance_metric = np.sqrt(scaled_circ_dist**2 + scaled_long_dist**2)
    
    # Core infarct is within certain distance of center
    core_infarct_mask = (
        (distance_metric <= 1.0) &  # Within the irregular elliptical boundary
        (phi_transmural <= infarct_transmural_max) & (phi_transmural >= infarct_transmural_min) &  # Transmural extent
        lv_mask  # Must be in LV territory
    )
    core_infarct_points = set(np.where(core_infarct_mask)[0])
    print(f"Found {len(core_infarct_points)} points in core infarct zone")
    
    # Border zone (outer perimeter around core)
    border_zone_mask = (
        (distance_metric <= border_zone_factor) & (distance_metric > 1.0) &  # Ring around core infarct
        (phi_transmural <= border_transmural_max) & (phi_transmural >= infarct_transmural_min) &  # Transmural extent
        lv_mask  # Must be in LV territory
    )
    border_zone_points = set(np.where(border_zone_mask)[0])
    print(f"Found {len(border_zone_points)} points in border zone")
    border_zone_points = set(np.where(border_zone_mask)[0])
    print(f"Found {len(border_zone_points)} points in border zone")
    
    # Step 5: Mark tetrahedra containing valid points
    print("Tagging tetrahedra...")
    n_tetrahedra = tetrahedra.shape[0]
    tetrahedra_tags = np.zeros(n_tetrahedra, dtype=int)
    
    # Process in batches to avoid memory issues
    batch_size = 10000
    for batch_start in tqdm(range(0, n_tetrahedra, batch_size), desc="Processing tetrahedra"):
        batch_end = min(batch_start + batch_size, n_tetrahedra)
        batch_tetrahedra = tetrahedra[batch_start:batch_end]
        
        # Check each tetrahedron in the batch
        for i, tet in enumerate(batch_tetrahedra):
            # Changed priority order: fast conducting endocardium (1) > core infarct (2) > border zone (3) > normal (0)
            # This preserves the fast conducting endocardium and prevents overlap with infarct regions
            if any(point_idx in valid_endo_points for point_idx in tet):
                tetrahedra_tags[batch_start + i] = 1  # Fast conducting endocardium
            elif any(point_idx in core_infarct_points for point_idx in tet):
                tetrahedra_tags[batch_start + i] = 2  # Core infarct
            elif any(point_idx in border_zone_points for point_idx in tet):
                tetrahedra_tags[batch_start + i] = 3  # Border zone
    
    # Count tagged tetrahedra
    num_endo = np.sum(tetrahedra_tags == 1)
    num_core = np.sum(tetrahedra_tags == 2)
    num_border = np.sum(tetrahedra_tags == 3)
    print(f"Tagged {num_endo} tetrahedra ({(num_endo/n_tetrahedra)*100:.2f}%) as fast conducting endocardium")
    print(f"Tagged {num_core} tetrahedra ({(num_core/n_tetrahedra)*100:.2f}%) as core infarct")
    print(f"Tagged {num_border} tetrahedra ({(num_border/n_tetrahedra)*100:.2f}%) as border zone")
    
    # Step 6: Write the modified .elem file
    print(f"Writing modified element file to {output_file}...")
    with open(output_file, 'w') as fout:
        # Write header (number of tetrahedra)
        fout.write(f"{n_tetrahedra}\n")
        
        # Process each tetrahedron and write to file
        for i in tqdm(range(n_tetrahedra), desc="Writing to file"):
            tet = tetrahedra[i]
            # Format: Tt node1 node2 node3 node4 tag
            line = f"Tt {tet[0]} {tet[1]} {tet[2]} {tet[3]} {tetrahedra_tags[i]}"
            fout.write(f"{line}\n")
    
    print(f"Successfully wrote modified element file to {output_file}")
    
    return tetrahedra_tags

In [27]:
# mesh_utils.tag_fast_conducting_endocardium(
#     points, 
#     tetrahedra, 
#     triangles, 
#     triangle_regions, 
#     phi_longitudinal, 
#     f"{data_path}/{prefix}.elem",
#     long_min = 0.2,
#     long_max = 0.85
# )

tag_lcx_infarct_and_endocardium(
    points, 
    tetrahedra, 
    triangles, 
    triangle_regions, 
    phi_transmural,
    phi_longitudinal, 
    phi_circumferential,
    f"{data_path}/{prefix}.elem",
    long_min=0.2,
    long_max=0.85,
    # Infarct location parameters
    infarct_center_circ=0.2,      # Center position around circumference (0-1, 0.25-0.35 for lateral wall)
    infarct_center_long=0.5,     # Center position along long axis (0-1, 0=apex, 1=base)
    # Infarct size parameters
    infarct_width_circ=0.04,      # Half-width in circumferential direction
    infarct_width_long=0.08,      # Half-width in longitudinal direction
    # Transmural extent parameters
    infarct_transmural_min=0.1,   # Minimum transmural extent (0=epicardium)
    infarct_transmural_max=0.7,   # Maximum transmural extent (1=endocardium)
    # Border zone parameters
    border_zone_factor=1.4,       # How much larger the border zone is compared to core
    border_transmural_max=0.9,    # Maximum transmural extent for border zone
    # Shape modifiers
    transmural_taper=0.8,         # How much the infarct tapers from epi to endo (0=no taper, 1=extreme taper)
    irregularity=1.2,             # Scale factor for boundary irregularity (0=smooth, 2=very irregular)
    random_seed=42,               # Seed for reproducible randomness
)

Starting LAD infarct and endocardium tagging...
Identifying points with phi_longitudinal between 0.2 and 0.85...
Found 356093 points with valid longitudinal coordinate
Identifying endocardial points...
Found 52674 points on the endocardium
Found 38338 points for fast conducting endocardium
Defining lateral wall (LCx) infarct region...
Identifying LV points...
Found 22982 points on the LV endocardium
Found 29692 points on the RV endocardium
LV center: [-42204.47786939   3287.49039165   3852.87911223]
RV center: [-9798.16758995 -3529.77555458  4007.89927438]
Identified 283614 points as LV territory
Found 1716 points in core infarct zone
Found 2256 points in border zone
Found 2256 points in border zone
Tagging tetrahedra...


Processing tetrahedra: 100%|██████████| 256/256 [00:04<00:00, 54.13it/s]


Tagged 259191 tetrahedra (10.14%) as fast conducting endocardium
Tagged 22439 tetrahedra (0.88%) as core infarct
Tagged 27692 tetrahedra (1.08%) as border zone
Writing modified element file to ../data/instance_001/instance_001.elem...


Writing to file: 100%|██████████| 2555157/2555157 [00:02<00:00, 979096.15it/s]

Successfully wrote modified element file to ../data/instance_001/instance_001.elem


array([0, 0, 0, ..., 1, 0, 0])

## Tag the fascicular sites and output to a .vtx file

In [ ]:
import numpy as np

def tag_fascicular_sites(points, phi_transmural, phi_longitudinal, phi_circumferential, 
                          triangles, triangle_regions):
    """
    Tag fascicular sites according to the Durrer-based model.
    
    Parameters:
    ----------
    points : numpy.ndarray
        Vertices of the mesh
    phi_transmural : numpy.ndarray
        Transmural coordinates (0 on epicardium, 1 on endocardium)
    phi_longitudinal : numpy.ndarray
        Longitudinal coordinates (1 on base, 0 on apex)
    phi_circumferential : numpy.ndarray
        Circumferential coordinates (wraps around from 0-1)
    triangles : numpy.ndarray
        Triangle indices
    triangle_regions : numpy.ndarray
        Region labels for each triangle (3 for LV endocardium, 4 for RV endocardium)
    
    Returns:
    -------
    is_fascicular_site : numpy.ndarray
        Boolean array indicating whether each point is a fascicular site
    fascicular_site_tag : numpy.ndarray
        Integer array with tags: 0 (not a site), 1 (LV anterior), 2 (LV posterior), 
        3 (LV/RV septal), 4 (RV moderator band)
    """
    # Constants based on the criteria
    transmural_depth = 1.0  # Just pick points directly on the endocardium
    disk_thickness = 0.2  # 0.5% of the ventricular wall
    
    # Initialize arrays
    num_points = len(points)
    is_fascicular_site = np.zeros(num_points, dtype=bool)
    fascicular_site_tag = np.zeros(num_points, dtype=int)
    
    # Define disk radius for endocardial extent
    disk_radius = 0.025  # Adjust this parameter as needed
    
    # Get indices of LV and RV endocardial points
    LV_points = np.array(list(set(triangles[triangle_regions == 3].flatten())))
    RV_points = np.array(list(set(triangles[triangle_regions == 4].flatten())))
    
    # Calculate centers for LV and RV
    LV_center = points[LV_points].mean(axis=0)
    RV_center = points[RV_points].mean(axis=0)
    
    # Calculate septal normal vector (from LV to RV)
    septal_normal = RV_center - LV_center
    septal_normal = septal_normal / np.linalg.norm(septal_normal)
    
    # Helper function to determine if a point is closer to LV or RV
    def is_closer_to_LV(point):
        mid_point = (LV_center + RV_center) / 2
        distance = np.dot(point - mid_point, septal_normal)
        return distance < 0  # Negative means closer to LV, positive means closer to RV
    
    # 1. LV anterior site
    lv_ant_circum = 0.25  # Around 1/4 of the way through circumferential coordinate
    lv_ant_longit = 0.5   # Middle of the LV along longitudinal axis
    
    lv_ant_mask = np.logical_and(
        np.abs(phi_transmural - transmural_depth) < disk_thickness/2,
        np.logical_and(
            (phi_circumferential - lv_ant_circum)**2 + (phi_longitudinal - lv_ant_longit)**2 < disk_radius**2,
            np.array([is_closer_to_LV(p) for p in points])
        )
    )
    
    is_fascicular_site = np.logical_or(is_fascicular_site, lv_ant_mask)
    fascicular_site_tag[lv_ant_mask] = 1
    
    # 2. LV posterior site
    lv_post_circum = 0.6  # Around 3/4 of the way through circumferential coordinate
    lv_post_longit = 0.5   # Middle of the LV along longitudinal axis
    
    lv_post_mask = np.logical_and(
        np.abs(phi_transmural - transmural_depth) < disk_thickness/2,
        np.logical_and(
            (phi_circumferential - lv_post_circum)**2 + (phi_longitudinal - lv_post_longit)**2 < disk_radius**2,
            np.array([is_closer_to_LV(p) for p in points])
        )
    )
    
    is_fascicular_site = np.logical_or(is_fascicular_site, lv_post_mask)
    fascicular_site_tag[lv_post_mask] = 2
    
    # 3. Septal sites (LV and RV)
    septal_anterior_circum = 0.95  # Anterior septal wall
    septal_longit = 0.5          # Middle along longitudinal axis
    
    septal_mask = np.logical_and(
        np.abs(phi_transmural - transmural_depth) < disk_thickness/2,
        (phi_circumferential - septal_anterior_circum)**2 + (phi_longitudinal - septal_longit)**2 < disk_radius**2
    )
    
    is_fascicular_site = np.logical_or(is_fascicular_site, septal_mask)
    fascicular_site_tag[septal_mask] = 3
    
    # 4. RV moderator band
    rv_mod_circum = 0.4      # Middle of RV free wall
    rv_mod_longit = 0.5      # Middle along longitudinal axis
    
    rv_mod_mask = np.logical_and(
        np.abs(phi_transmural - transmural_depth) < disk_thickness/2,
        np.logical_and(
            (phi_circumferential - rv_mod_circum)**2 + (phi_longitudinal - rv_mod_longit)**2 < disk_radius**2,
            np.logical_not(np.array([is_closer_to_LV(p) for p in points]))
        )
    )
    
    is_fascicular_site = np.logical_or(is_fascicular_site, rv_mod_mask)
    fascicular_site_tag[rv_mod_mask] = 4
    
    return is_fascicular_site, fascicular_site_tag


is_fascicular_site, fascicular_site_tag = tag_fascicular_sites(points, phi_transmural, phi_longitudinal, phi_circumferential, triangles, triangle_regions)

mesh_utils.visualize_fascicular_sites(
    points, triangles, triangle_regions,
    is_fascicular_site, fascicular_site_tag, 
    sphere_scale=1000,
)

immediate_stim = np.logical_and(is_fascicular_site, np.logical_or(fascicular_site_tag == 1, np.logical_or(fascicular_site_tag == 2, fascicular_site_tag == 3)))
mesh_utils.save_fascicular_sites_to_vtx(immediate_stim, output_filename=f"{data_path}/immediate_stim.vtx")
delayed_stim = np.logical_and(is_fascicular_site, fascicular_site_tag == 4)
mesh_utils.save_fascicular_sites_to_vtx(delayed_stim, output_filename=f"{data_path}/delayed_stim.vtx")

Original points: 478820
Original triangles: 199902
Number of fascicular sites: 307
Endocardium triangles: 104504
Displaying all 307 fascicular sites


/Users/jamesmcgreivy/miniconda3/envs/openCARP/lib/python3.10/site-packages/pyvista/core/filters/data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:53029/index.html?ui=P_0x323624400_6&reconnect=auto" class="pyvista…

Saving 221 fascicular site indices to ../data/instance_001/immediate_stim.vtx
Successfully saved fascicular sites to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/immediate_stim.vtx
Saving 86 fascicular site indices to ../data/instance_001/delayed_stim.vtx
Successfully saved fascicular sites to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/delayed_stim.vtx


'/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/delayed_stim.vtx'

(array([[-56017.29202271,  23638.37051392,  22255.58662415],
        [-55755.47027588,  23849.31755066,  21746.54579163],
        [-56104.94232178,  23719.61212158,  21729.83551025],
        ...,
        [ 14146.57688141,  12142.09842682,  19588.19580078],
        [-46888.35525513,  19900.38490295, -25976.59301758],
        [-64681.43463135,  11666.8586731 , -12088.70124817]]),
 array([0.        , 0.        , 0.        , ..., 0.8383389 , 0.5232232 ,
        0.08157784]),
 array([0.79719543, 0.79233015, 0.79219085, ..., 0.8636715 , 0.32573876,
        0.5043144 ]),
 array([0.49104473, 0.49286485, 0.49013183, ..., 0.38543484, 0.46585423,
        0.32361218]))

##  Extra utility function -- for visualizing the scalar fields

In [12]:
mesh_utils.visualize_phi(points, phi_transmural, subsample_factor=1, point_size=5)

Original points: 478820
Subsampling points from 478820 to 50000


Widget(value='<iframe src="http://localhost:59447/index.html?ui=P_0x1570fe8c0_2&reconnect=auto" class="pyvista…